[![imagenes/pythonista.png](imagenes/pythonista.png)](https://pythonista.io)

El formato PDF es un [estándar internacional](https://www.iso.org/standard/51502.html) para la creación de documentos que pueden ser desplegados o impresos de forma idéntica independientemente del dispositivo o aplicación que acceda a dichos documentos.

El énfasis del formato PDF reside en la correcta legibilidad de la información para los seres humanos, poniendo en segundo término la estructura y orden de la información que contiene.


## Creación de documentos PDF a partir de HTML con *wkhtmltopdf*.

El proyecto https://wkhtmltopdf.org/ ofrece una herramienta capaz de transformar código HTML en documentos PDF. Dicha herramienta se ejecuta directamente como un comando del sistema con la sintaxis.

```
wkhtmltopdf <archivo HTML de origen> <archivo PDF resultante> <parámetros>
```

### Restricciones. 

El paquete *wkhtmltopdf* requiere del uso de un servidor X y la biblioteca Qt. En el caso de las instalaciones en sistemas Windows, los binarios incluyen lo necesario para poder ejecutar la herramienta desde una terminal. Del mismo modo, los sistemas de escritorio de Linux también cuentan con un entrono gráfico gestionado por un servidor X. Sin embargo, es muy común que un servidor Linux no cuente con un servidor X instalado.

Para que *pdfkit* pueda funcionar, es necesario que *wkhtmltopdf* se encuentre en una ruta conocida, por lo que en el caso de Windows, es necesario añadir esta ruta a la variable de entorno *PATH*.

## Instalación de *wkhtmltopdf* junto con un servidor virtual de X en CentOS 7.

La máquina virtual que fue descargada del sitio http://pythonista.io no cuenta con un servidor X, por lo que es necesario instalarlo al igual que el paquete *wkhtmltopdf*.

**Nota:**
Este ejercicio representa el caso menos favorable. Si se utiliza esta herramienta desde un sistema Windows o desde un entorno gráfico en Linux, el uso de esta herramienta es mucho más simple.

### Instalación y configuración del servidor X virtual.
El servidor X virtual o *xvfb* por las siglas en inglés de "X Virtual FrameBuffer" es una variante de un servidor X que realiza las mismas opraciones que uno convencional, pero no se despliega en pantalla. Se utiliza para realizar opraciones que requieren de un entrono gráfico en un sistema GNU/Linux.

#### Instalación del paquete.

En este caso se utilizará el gestor de paquetes, *yum*.

In [ ]:
!sudo yum install xorg-x11-server-Xvfb -y

#### Creación del archivo de definición de servicio.
Una vez instalado, se define el servicio añadiendo el archivo [*src/xvfb.service*](src/xvfb.service) en */etc/systemd/system/* como superusuario.

Dicho archivo contiene el siguiente código:

``` bash
[Unit]
Description=Virtual Frame Buffer X Server
After=network.target

[Service]
ExecStart=/usr/bin/Xvfb :42 -screen 0 1024x768x24 -ac +extension GLX +render -noreset

[Install]
WantedBy=multi-user.target
```

In [ ]:
!sudo cp src/xvfb.service /etc/systemd/system/

#### Habilitación y puesta en marcha del servicio.
CentOS 7 utiliza el comando *systemctl* para gestionar servicios.

In [ ]:
!sudo systemctl enable xvfb

In [ ]:
!sudo systemctl start xvfb

Para comprobar que el servidor está corriendo ejecute lo siguiente:

In [ ]:
!sudo systemctl status xvfb

**Nota:** Este procedimiento sólo se requiere hace una vez. Una vez instalado, configurado y habilitado, el servicio iniciará junto con el servidor.

### Instalación y configuración de *wkhtmltopdf*.

#### Instalación del paquete con *yum*.

In [ ]:
!sudo yum install wkhtmltopdf -y

#### Creación de un script que permita acceder al servidor virtual.

Es necesario que wkthmltopdf pueda acceder al servidor virtual, por lo que se debe de crear un script que incluya al comando *xvfb-run* el cual le indica al sistema que la instrucción hará uso de un servidor X virtual.

El script [src/wkhtmltopdf.sh](src/wkhtmltopdf.sh) contiene el siguiente código:

``` bash
#!/bin/bash
xvfb-run -a --server-args="-screen 0, 1280x800x24" /bin/wkhtmltopdf -q $*
```

El script en cuestión debe de contar con permisos de ejecución, lo cual se realiza con el siguiente comando. 

In [ ]:
!chmod +x src/wkhtmltopdf.sh

#### Ejecución del script.

**Nota:** Este es un ejemplo de ejecución en la línea de comandos. No es código en Python. 

La siguiente celda ejecutará el script [*src/wkhtmltopdf.sh*]([src/wkhtmltopdf.sh]) 
* Se conectará a *http://google/com*.
* El contenido será convertido a PDF y guarado en [*google.pdf*](google.pdf).

In [ ]:
!src/wkhtmltopdf.sh http://google.com google.pdf

## Instalación de *wkhtmltopdf* en Windows.

En el caso de Windows sólo es necesario [descargar del sitio](https://wkhtmltopdf.org/downloads.html) la versión *MSVC 2015* y ejecutar el instalador.

Por lo general, el ejecutable se instala en *C:\Program Files\wkhtmltopdf\bin\wkhtmltopdf.exe*

**Ejemplo:**

La siguiente celda ejecutará *wkhtmltopdf.exe* en la ruta indicada. 
* Se conectará a *http://google/com*.
* El contenido será convertido a PDF y guarado en [*google.pdf*](google.pdf).

In [ ]:
!"C:\Program Files\wkhtmltopdf\bin\wkhtmltopdf" "https://google.com" "google.pdf"

## El paquete *pdfkit*.

El paquete [*pfdkit*](https://pypi.org/project/pdfkit/#description) es una implementación de *wkhtmltopdf* para Python.

In [ ]:
!pip install pdfkit

In [ ]:
import pdfkit

### Funciones de *pdfkit* dependiendo la fuente de los datos.

Es posible tranformar documentos HTML desde diversas fuentes.

#### Si el código HTML es ingresado como una cadena de caracteres se utiliza la función *pdfkit.fromstring()* con la siguiente  sintaxis:
```
pdfkit.fromstring(<objeto tipo str>, '<ruta del archivo resultante>', <argumentos>)
```

#### Si el código HTML proviene de un archivo local, se utiliza la función *pdfkit.fromfile()* con la siguiente  sintaxis:
```
pdfkit.fromfile('<ruta>', '<ruta del archivo resultante>', <argumentos>)
```

#### Si el código HTML proviene de un recurso en línea, se utiliza la función *pdfkit.fromulr()* con la siguiente  sintaxis:
```
pdfkit.fromurl('<URL>', '<ruta del archivo resultante>', <argumentos>)
```

### El objeto *pdfkit.configuration*.
Para poder utilizar *pdfkit* es necesario que *wkhtmltopdf* esté instalado y que la ruta donde se encuentra el ejecutable sea conocida.

El objeto *pdfkit.configuration* permite ingresar diversos parámetros de configuración y en este caso se puede indicar dónde se encuentra el archivo ejecutable de *wkhtmltopdf* con la siguiente sintaxis:

```python
pdfkit.configuration(wkhtmltopdf='<ruta>')
```
En caso de que el ejecutable de *wkhtmltopdf* sea accesible desde el sistema, no es necesario utilizar este objeto.

**Ejemplo:** 

En este caso, el ejecutable que utilizará *pdfkit* corresponde al script localizado en [src/wkhtmltopdf.sh](src/wkhtmltopdf.sh)

In [ ]:
conf = pdfkit.configuration(wkhtmltopdf='src/wkhtmltopdf.sh')

En caso de haber instalado *wkhtmltopdf* en Windows, la configuración sería:

In [ ]:
conf = pdfkit.configuration(wkhtmltopdf='C:\\Program Files\\wkhtmltopdf\\bin\\wkhtmltopdf.exe')

**Ejemplos:**

Se convertirán archivos HTML localizados en archivos locales y en línea mediante *pdfkit*.

En este caso se convertirá el contenido de [html/menu.html](html/menu.html) y se guardará en [menu.pdf](menu.pdf).

In [ ]:
pdfkit.from_file('html/menu.html', 'menu.pdf', configuration=conf)

En este caso se convertirá el contenido de https://pythonista.io/cursos/py101/ciclos-con-while-e-interrupciones-de-flujo y se guardará en [ciclos.pdf](ciclos.pdf).

In [ ]:
pdfkit.from_url('https://pythonista.io/cursos/py101/ciclos-con-while-e-interrupciones-de-flujo',
                'ciclos.pdf', configuration=conf)

### El parámetro *options*.

Es posible modificar el tipo y orientación de la página, el tamaño de los márgenes y algunas otras características del documento PDF resultante mediante algunas opciones de *wkhtmltopdf*, las cuales pueden ser consultadas en https://github.com/worlduniting/bookshop/wiki/wkhtmltopdf-options

En el caso de *pdfkit* estas opciones deben de estar contenidas en un objeto tipo *dict*.

**Ejemplo:**

Se creará nuevamente el archivo [menu.pdf](menu.pdf) con algunos ajustes a los márgenes e indicando el tipo de tamano de papel.

In [ ]:
options = {
    'page-size': 'Letter',
    'margin-top': '0.75in',
    'margin-right': '0.75in',
    'margin-bottom': '0.75in',
    'margin-left': '0.75in',
    'encoding': "UTF-8"
}

In [ ]:
pdfkit.from_file('html/menu.html', 'menu.pdf', configuration=conf, options=options)

## Lectura de archivos PDF.

La extracción de datos a partir de archivos PDF no es simple y en muchos casos tampoco es infalible. Es por ello por lo que se detallará el uso de las siguientes herramientas.

* [*pdfminer.six*](https://github.com/pdfminer/pdfminer.six).
* [*pdftotext*](https://github.com/jalan/pdftotext).
* [*PyPDF2*](https://mstamy2.github.io/PyPDF2/).

### El paquete *pdfminer.six*.

El paquete *pdfminer.six* es una implementación compatible con Python 3 de *pdfminer*. Es una herramienta especializada para la extracción de texto muy popular. 

Tiene la  ventaja de acceder  a porciones específicas de un documento PDF y gestionarlas en memoria.

Los paquetes *pdfminer* y *pdfminer.six* cuentan con múltiples clases especializadas en gestionar, seleccionar, obtener, interpretar y convertir porciones específicas de un documento PDF.

Para mayor información consultar https://pdfminer-docs.readthedocs.io/

#### Extracción de un documento PDF con *pdfminer.six*.

A continuacion se creará una función capaz de extraer el texto de las pág
inas indicadas desde un archivo PDF.

* Se instala el paquete *pdfminer.six*.

In [ ]:
! pip install pdfminer.six

* La clase *io.StringIO* del paquete *io* de Python permite acceder a un recurso en el disco y guardarlo en memoria. De este modo podemos guardar porciones del archivo PDF en un *buffer*.

In [ ]:
from io import StringIO

El módulo *pdfminer.pdfinterp* contiene a las clases:

* *PDFResourceManager*, cuyas instancias son capaces de gestionar el flujo de recursos de un documento PDF a los que *pdfminer* accede. En este caso, se encargará de  gestionar el flujo entre un archivo y el buffer en memoria.
* *PDFPageInterpreter*, cuyas instancias tienen la capacidad de acceder e interpetar el contenido de un recurso proveniente de un archivo PDF.

In [ ]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter

* El módulo *pdfminer.converter* contiene varias clases capaces de convertir contenidos específicos de in documento PDF, talkes como XML, HTML o texto.

* En este caso, se utilizará la clase *TextConverter*.

In [ ]:
from pdfminer.converter import TextConverter

Los archivos PDF incluyen por lo general una plantilla (layout) específica del documento que puede definir el tamaño de página, orientacion, numeración, márgenes, etc.

* El módulo *pdfminer.layout* permite acceder a la información de estas plantillas y la clase *pdfminer.layout.LAParams*, se encarga de obtener los parámetros definidos en la plantilla de un documento PDF.

In [ ]:
from pdfminer.layout import LAParams

Los documentos PDF está compuestos por páginas cuyas características están definidas en los parámetros de la plantilla correspondiente.

* La clase *pdfminer.pdfpage.PDFPage* crea objetos que emulan a una página del documento PDF.

In [ ]:
from pdfminer.pdfpage import PDFPage

* La función *convierte()* permite extraer el texto de las páginas enumeradas dentro de un objeto iterable que se ingresa como el argumento *paginas* a partir del archivo definido en el parámetro *ruta*. En caso de no indicarlos, extraerá el texto de todas las páginas del documento.

In [ ]:
def convierte(ruta, paginas=None):
    if not paginas:
        numero_paginas = set()
    else:
        numero_paginas = set(paginas)
    salida = StringIO()
    gestor = PDFResourceManager()
    convertidor = TextConverter(gestor, salida, laparams=LAParams())
    interprete = PDFPageInterpreter(gestor, convertidor)

    archivo = open(ruta, 'rb')
    for pagina in PDFPage.get_pages(archivo, numero_paginas):
        interprete.process_page(pagina)
    archivo.close()
    convertidor.close()
    texto = salida.getvalue()
    salida.close
    return texto 

**Ejemplos:**

* Se obtendrá el texto de la segunda página del archivo [*menu.pdf*](menu.pdf) que fue creado previamente.

In [ ]:
convierte('menu.pdf', paginas=[1])

* Se obtendrá el texto completo del archivo [*menu.pdf*](menu.pdf).

In [ ]:
convierte('menu.pdf')

### El paquete *pdftotext*.

Este paquete es una implementación de la biblioteca [*poppler*](https://poppler.freedesktop.org/), perteneciente a [XpdfReader](https://www.xpdfreader.com/). Este paquete corre en Linux, UNIX y MacOS X.

In [ ]:
!sudo yum install poppler-cpp-devel -y

In [ ]:
!pip install pdftotext

In [ ]:
import pdftotext

La clase *pdftotext.PDF* permite crear un objeto iterador que contienen el texto extraido de cada página de un documento PDF.

In [ ]:
with open("menu.pdf", "rb") as archivo:
    pdf = pdftotext.PDF(archivo)

In [ ]:
type(pdf)

In [ ]:
for page in pdf:
    print(page)

### El paquete *PyPDF2*.

Existen diversos proyectos enfocados a la gestión de archivos en formato PDF. El paquete *PyPDF2* permite no sólo leer sino escribir, concatenar y modificar archivos PDF. 

Para los fines de esta sección sólo se utilizará la clase *PyPDF2.PdfFileReader* para leer el contenido de un archivo PDF.

Para saber mas sobre este paquete puede consultar en https://pythonhosted.org/PyPDF2/

In [ ]:
!pip install PyPDF2

#### La clase *PyPDF2.PdfFileReader*

Esta clase crea un objeto que posee atributos correspontientes a las características, estructuras y contenidos de un archivco PDF. Para crear un objeto a partir de esta clase su tiliza la siguiente sintaxis:

```
PyPDF2.PdfFileReader('<ruta del archivo>')
```
Aún cuando el paquete es muy versatil, en muchos casos no es tan eficiente para extraer texto como las herramientas mencionadas anteriormente.

In [ ]:
from PyPDF2 import PdfFileReader

In [ ]:
with open('menu.pdf', 'rb') as archivo:
    pdf = PdfFileReader(archivo)
    print('Información del documento:', pdf.documentInfo)
    print('Número de páginas:', pdf.numPages)
    pagina = pdf.getPage(0)
    print('Contenido de la pagina 1:\n', pagina.extractText())

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2018.</p>